In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 22.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=2606cc594f9cfc5e126f9d57daa48e78ac987b5d9dbebf4141ac54755cfe6754
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
pip install keras-tcn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.4 MB/s eta 0:00:00


In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'tcn_aguro',
    'metric': {
        'goal': 'minimize', 
        'name': 'validation_loss'
		},
    'parameters': {
        'batch_size': {'value': 512},
        'epochs': {'value': 500},
        'lr': {'max': 0.1, 'min': 0.0001},
        'dropout_rate': {'values': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]},
        'optimizer': {'value': 'adam'},
        'nb_filters': {'values': [16, 32, 64]},
        'kernel_size': {'values': [3, 6, 8, 12]}
     }
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pandas==1.4.4

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 96.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
import pandas as pd
# from tsai.all import SlidingWindowPanel
import numpy as np

from sklearn.model_selection import train_test_split

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
def norm_df(df,cols):
  for col in cols:
    df[col] = df[col] / df[col].max()
  return df

In [ ]:
df_shift7 = pd.read_pickle('/content/drive/MyDrive/Master/DSP/merge_df_shift_7.pickle')
df_shift7 = df_shift7[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',  'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]
df_shift14 = pd.read_pickle('/content/drive/MyDrive/Master/DSP/merge_df_shift_14.pickle')
df_shift14 = df_shift14[['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg',
        'prcp','NDVI_mean_shift', 'tavg_shift', 'prcp_shift']]

In [ ]:
df_shift7 = norm_df(df_shift7, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift14 = norm_df(df_shift14, ['tavg','prcp','tavg_shift','prcp_shift'])
df_shift7.dropna(axis = 0, inplace = True)
df_shift14.dropna(axis = 0, inplace = True)

In [ ]:
df_shift14.columns

Index(['date', 'NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'name', 'tavg', 'prcp',
       'NDVI_mean_shift', 'tavg_shift', 'prcp_shift'],
      dtype='object')

In [ ]:
x_cols = ['NDVI_mean', 'NDMI_mean', 'NBSI_mean', 'tavg_shift', 'prcp_shift']

In [ ]:
sliding_window_X, sliding_window_y = SlidingWindowPanel(window_len=14, unique_id_cols=[
                                                        'name'], get_x=x_cols, get_y=['NDVI_mean_shift'], stride=1, horizon = 14)(df_shift14)


processing data...


...data processed
concatenating X...
...X concatenated
concatenating y...
...y concatenated


In [ ]:
sliding_window_y.shape

(48607, 14)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sliding_window_X, sliding_window_y,
                                                    random_state=42, shuffle=False, test_size=0.2)

In [ ]:
X_train.shape

(38885, 5, 14)

In [ ]:
X_train = np.moveaxis(X_train, 1, -1)
X_train.shape

(38885, 14, 5)

In [ ]:
X_test = np.moveaxis(X_test, 1, -1)
X_test.shape

(9722, 14, 5)

In [ ]:
y_train.shape

(38885, 14)

In [ ]:
np.save('X_train14', X_train)

In [ ]:
np.save('X_test14', X_test)

In [ ]:
np.save('y_train', y_train)
np.save('y_test', y_test)

In [ ]:
import tensorflow as tf
from tcn import TCN, compiled_tcn
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input

In [ ]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
y_train.shape

In [ ]:
import tensorflow.keras.backend as K
def rmse(y_true,y_pred):
  return K.sqrt(K.mean(K.square(y_true - y_pred)))

In [ ]:
def compile_model(swp_X, nb_filters, kernel_size, dilations, dropout_rate, y_train):

    # output tcn2
    nb_features = swp_X.shape[2]
    nb_outputs = y_train.shape[1]

    model = Sequential()
    model.add(LSTM(nb_filters, return_sequences=True,
                  input_shape=(swp_X.shape[1], nb_features), dropout=dropout_rate))  # returns a sequence of vectors of dimension 32
    model.add(LSTM(nb_filters, return_sequences=True, dropout=dropout_rate))  # returns a sequence of vectors of dimension 32
    model.add(LSTM(nb_filters))  # return a single vector of dimension 32
    model.add(Dense(nb_outputs, activation='softmax'))

    return model

In [ ]:
def get_optimizer(lr=1e-3, optimizer="adam"):
    "Select optmizer between adam and sgd with momentum"
    if optimizer.lower() == "adam":
        return tf.keras.optimizers.Adam(learning_rate=lr)
    if optimizer.lower() == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.1)
    if optimizer.lower() == "adadelta":
        return tf.keras.optimizers.Adadelta(learning_rate=lr)


def train(X_train, y_train, batch_size=512, epochs=500, lr=1e-3, optimizer='adam', dropout_rate=0.1, nb_filters=32, kernel_size=3, dilations=[
            1, 2, 4, 8, 16, 32, 64]):  

    model = compile_model(X_train, nb_filters, kernel_size, dilations, dropout_rate, y_train)
    
    # Compile model like you usually do.
    tf.keras.backend.clear_session()
    model.compile(loss=root_mean_squared_error, 
                  optimizer=get_optimizer(lr, optimizer), 
                  metrics=['mae', root_mean_squared_error])

    # callback setup
    wandb_callbacks = [
        WandbCallback(),
        EarlyStopping(patience=15),
        ReduceLROnPlateau(patience=10)
        #WandbModelCheckpoint(filepath="my_model_{epoch:02d}")
    ]

    model.fit(X_train, 
              y_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_split=0.2, 
              callbacks=wandb_callbacks)

In [ ]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults, reinit=True):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration, if any
        wandb.config.architecture_name = "TCN"
        wandb.config.dataset_name = "Sentinel_2_ndvi"

        train(X_train,
              y_train,
              wandb.config.batch_size, 
              wandb.config.epochs,
              wandb.config.lr,
              wandb.config.optimizer,
              wandb.config.dropout_rate,
              wandb.config.nb_filters,
              wandb.config.kernel_size
              )

In [ ]:
X_train = np.load('X_train14.npy')
X_test = np.load('X_test14.npy')
y_train = np.load('y_train14.npy')
y_test = np.load('y_test14.npy')

In [ ]:
model = compile_model(X_train, 64, 3, [1, 2, 4, 8, 16, 32, 64], 0.1, y_train)

In [ ]:
run = wandb.init(project='LSTM_aguro', entity='gijstimmerij', reinit=True)

wandb: Currently logged in as: gijstimmerij. Use `wandb login --relogin` to force relogin


In [ ]:
model.compile(loss=root_mean_squared_error,
              optimizer='adam', metrics=['mae', root_mean_squared_error]
              )

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 14, 64)            17920     
                                                                 
 lstm_8 (LSTM)               (None, 14, 64)            33024     
                                                                 
 lstm_9 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 14)                910       
                                                                 
Total params: 84,878
Trainable params: 84,878
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_train = np.load('X_train14.npy')
X_test = np.load('X_test14.npy')
y_train = np.load('y_train14.npy')
y_test = np.load('y_test14.npy')

In [ ]:
wandb_callbacks = [WandbCallback(),
                   EarlyStopping(patience=15),
                   ReduceLROnPlateau(patience=10)
]

model.fit(X_train, 
          y_train, 
          batch_size=512, 
          epochs=200, 
          validation_split=0.2, 
          callbacks=wandb_callbacks)

Epoch 1/200
60/61 [============================>.] - ETA: 0s - loss: 0.6332 - mae: 0.5987 - root_mean_squared_error: 0.6332

wandb: Adding directory to artifact (/content/wandb/run-20230124_131711-qodyytxu/files/model-best)... Done. 0.0s


61/61 [==============================] - 17s 211ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 2/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 3/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 4/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7243 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 5/200
58/61 [===========================>..] - ETA: 0s - loss: 0.6332 - mae: 0.5987 - root_mean_squared_error: 0.6332

wandb: Adding directory to artifact (/content/wandb/run-20230124_131711-qodyytxu/files/model-best)... Done. 0.0s


61/61 [==============================] - 11s 190ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6330 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 6/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 7/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 8/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7243 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 0.0010
Epoch 9/200
61/61 [==============================] - 1s 10ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0

wandb: Adding directory to artifact (/content/wandb/run-20230124_131711-qodyytxu/files/model-best)... Done. 0.0s


61/61 [==============================] - 12s 192ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7279 - lr: 0.0010
Epoch 12/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6330 - mae: 0.5985 - root_mean_squared_error: 0.6331 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 1.0000e-04
Epoch 13/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6330 - mae: 0.5985 - root_mean_squared_error: 0.6330 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 1.0000e-04
Epoch 14/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6331 - mae: 0.5985 - root_mean_squared_error: 0.6330 - val_loss: 0.7242 - val_mae: 0.7126 - val_root_mean_squared_error: 0.7280 - lr: 1.0000e-04
Epoch 15/200
61/61 [==============================] - 1s 11ms/step - loss: 0.6330 - mae: 0.5985 - root_mean_squared_error: 0.63

In [ ]:
run.finish()

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▇██▆▇▆▇▆▃▅▃▃▄▄▄▄▃▄▃▃▄▃▁▂▄
mae,▆▆▇█▆▆▇█▅▅▆▂▃▂▃▂▃▁▁▁▁▅▃▃▂▄
root_mean_squared_error,▅█▄█▄▆▅▆█▂█▇▃▂▂▆▂▄▃▁▅▃▆▃▂▄
val_loss,▅▅▆█▅▆▅█▅▆▁▄▂▂▃▄▂▂▂▃▄▄▄▃▃▃
val_mae,▆▅▅▆▄▅▄█▄▃▁▄▂▃▃▄▂▂▃▄▅▄▄▄▄▄
val_root_mean_squared_error,▄▅▆█▅▅▅▇▅▆▁▄▂▂▃▄▂▂▃▃▄▄▄▃▃▃
best_epoch,10
best_val_loss,0.7242
epoch,25
loss,0.63304


In [ ]:
sweep_id = wandb.sweep(sweep_configuration, project="aguro_tcn")

Create sweep with ID: 9vw7gd6y
Sweep URL: https://wandb.ai/dsp_aguro/aguro_tcn/sweeps/9vw7gd6y


In [ ]:
wandb.agent(sweep_id, function=sweep_train, count=100)

wandb: Agent Starting Run: oag9879x with config:
wandb: 	batch_size: 512
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 500
wandb: 	kernel_size: 12
wandb: 	lr: 0.0717458955462187
wandb: 	nb_filters: 16
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/500
 6/61 [=>............................] - ETA: 4s - loss: 0.9195 - mae: 0.7334 - root_mean_squared_error: 0.9195

61/61 [==============================] - ETA: 0s - loss: 0.3031 - mae: 0.2348 - root_mean_squared_error: 0.3028

wandb: Adding directory to artifact (/content/wandb/run-20230124_115339-oag9879x/files/model-best)... Done. 0.0s


...

61/61 [==============================] - 18s 305ms/step - loss: 0.2173 - mae: 0.1654 - root_mean_squared_error: 0.2173 - val_loss: 0.1650 - val_mae: 0.1422 - val_root_mean_squared_error: 0.1670 - lr: 0.0780
Epoch 11/500
61/61 [==============================] - 7s 118ms/step - loss: 0.2170 - mae: 0.1650 - root_mean_squared_error: 0.2170 - val_loss: 0.1823 - val_mae: 0.1607 - val_root_mean_squared_error: 0.1846 - lr: 0.0780
Epoch 12/500
61/61 [==============================] - 7s 114ms/step - loss: 0.2170 - mae: 0.1648 - root_mean_squared_error: 0.2170 - val_loss: 0.1737 - val_mae: 0.1515 - val_root_mean_squared_error: 0.1758 - lr: 0.0780
Epoch 13/500
61/61 [==============================] - 7s 115ms/step - loss: 0.2169 - mae: 0.1649 - root_mean_squared_error: 0.2169 - val_loss: 0.1799 - val_mae: 0.1581 - val_root_mean_squared_error: 0.1821 - lr: 0.0780
Epoch 14/500
61/61 [==============================] - 7s 115ms/step - loss: 0.2174 - mae: 0.1653 - root_mean_squared_error: 0.2174 - val

wandb: Adding directory to artifact (/content/wandb/run-20230124_120411-ldxxab6s/files/model-best)... Done. 0.0s


61/61 [==============================] - 17s 280ms/step - loss: 0.2172 - mae: 0.1653 - root_mean_squared_error: 0.2171 - val_loss: 0.1606 - val_mae: 0.1373 - val_root_mean_squared_error: 0.1625 - lr: 0.0780
Epoch 16/500
61/61 [==============================] - 7s 114ms/step - loss: 0.2173 - mae: 0.1652 - root_mean_squared_error: 0.2172 - val_loss: 0.1751 - val_mae: 0.1530 - val_root_mean_squared_error: 0.1772 - lr: 0.0780
Epoch 17/500
61/61 [==============================] - 7s 114ms/step - loss: 0.2172 - mae: 0.1650 - root_mean_squared_error: 0.2172 - val_loss: 0.1714 - val_mae: 0.1491 - val_root_mean_squared_error: 0.1735 - lr: 0.0780
Epoch 18/500
61/61 [==============================] - 7s 115ms/step - loss: 0.2171 - mae: 0.1650 - root_mean_squared_error: 0.2171 - val_loss: 0.1658 - val_mae: 0.1430 - val_root_mean_squared_error: 0.1678 - lr: 0.0780
Epoch 19/500
61/61 [==============================] - 7s 115ms/step - loss: 0.2176 - mae: 0.1655 - root_mean_squared_error: 0.2177 - val

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mae,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▅▅▅▃▅▄▆▂▆▄▆▃▁▅▄▂▅▅▇▆▆▅▄▆▄▅▅▄
val_mae,█▃▅▅▅▃▅▄▆▂▆▄▆▃▁▅▄▂▅▅▇▆▆▆▄▆▄▅▅▄
val_root_mean_squared_error,█▃▅▅▅▃▅▄▆▂▆▄▆▃▁▅▄▂▅▅▇▆▆▆▄▆▄▅▅▄
best_epoch,14
best_val_loss,0.16063
epoch,29
loss,0.21677


wandb: Agent Starting Run: w9wsx402 with config:
wandb: 	batch_size: 512
wandb: 	dropout_rate: 0.7
wandb: 	epochs: 500
wandb: 	kernel_size: 6
wandb: 	lr: 0.06097932802114634
wandb: 	nb_filters: 32
wandb: 	optimizer: adam


In [ ]:
model.evaluate(X_test, y_test)

304/304 [==============================] - 2s 6ms/step - loss: 0.1350 - mae: 0.1168 - root_mean_squared_error: 0.1350


[0.13501565158367157, 0.11678452789783478, 0.1349869966506958]

In [ ]:
preds = model.predict(X_test)

304/304 [==============================] - 2s 5ms/step


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
preds[-1]

NameError: ignored